In [1]:
import numpy as np
import pandas as pd

In [3]:
loan_data_inputs_train = pd.read_csv('loan_data_inputs_train.csv',index_col=0)
loan_data_outputs_train = pd.read_csv('loan_data_targets_train.csv',index_col=0,header=None)
loan_data_inputs_test = pd.read_csv('loan_data_inputs_test.csv',index_col=0)
loan_data_outputs_test = pd.read_csv('loan_data_targets_test.csv',index_col=0,header=None)

In [4]:
loan_data_inputs_train.head(10)

,Unnamed: 0.1,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,...,dti:21.7-22.4,dti:22.4-35,dti:>35,mths_since_last_record:Missing,mths_since_last_record:0-2,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>86
427211,427211,12796369,14818505,24000,24000,24000.000000,36 months,8.90,762.08,A,...,1,0,0,1,0,0,0,0,0,0
206088,206088,1439740,1691948,10000,10000,10000.000000,36 months,14.33,343.39,C,...,0,0,0,1,0,0,0,0,0,0
136020,136020,5214749,6556909,20425,20425,20425.000000,36 months,8.90,648.56,A,...,0,0,0,1,0,0,0,0,0,0
412305,412305,13827698,15890016,17200,17200,17200.000000,36 months,16.59,609.73,D,...,0,0,0,1,0,0,0,0,0,0
36159,36159,422455,496525,8400,8400,7450.000000,36 months,12.84,282.40,C,...,0,1,0,1,0,0,0,0,0,0
260400,260400,34191891,36855167,20000,20000,20000.000000,36 months,7.49,622.04,A,...,0,0,0,0,0,0,0,1,0,0
29626,29626,520869,673425,5000,5000,4848.011841,60 months,10.62,107.77,B,...,0,0,0,1,0,0,0,0,0,0
117003,117003,6105242,7587376,24000,24000,24000.000000,60 months,20.31,640.01,D,...,0,1,0,1,0,0,0,0,0,0
378696,378696,17694110,19826620,20000,20000,20000.000000,36 months,9.17,637.58,B,...,0,0,0,1,0,0,0,0,0,0
159555,159555,3636501,4588773,21000,21000,21000.000000,60 months,15.31,503.02,C,...,0,0,0,1,0,0,0,0,0,0


In [5]:
loan_data_outputs_train.head()

,1
0,
427211,1
206088,1
136020,1
412305,0
36159,0


In [6]:
loan_data_outputs_train.describe()

,1
count,373028.000000
mean,0.888054
std,0.315300
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [7]:
loan_data_inputs_train.shape

(373028, 323)

In [8]:
loan_data_outputs_train.shape

(373028, 1)

In [9]:
loan_data_inputs_test.shape

(93257, 323)

In [10]:
loan_data_outputs_test.shape

(93257, 1)

## Selecting the features

In [11]:
# Here we select a limited set of input variables in a new dataframe.
inputs_train_with_ref_cat = loan_data_inputs_train.loc[: , ['grade:A',
'grade:B',
'grade:C',
'grade:D',
'grade:E',
'grade:F',
'grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'home_ownership:OWN',
'home_ownership:MORTGAGE',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'addr_state:NM_VA',
'addr_state:NY',
'addr_state:OK_TN_MO_LA_MD_NC',
'addr_state:CA',
'addr_state:UT_KY_AZ_NJ',
'addr_state:AR_MI_PA_OH_MN',
'addr_state:RI_MA_DE_SD_IN',
'addr_state:GA_WA_OR',
'addr_state:WI_MT',
'addr_state:TX',
'addr_state:IL_CT',
'addr_state:KS_SC_CO_VT_AK_MS',
'addr_state:WV_NH_WY_DC_ME_ID',
'verification_status:Not Verified',
'verification_status:Source Verified',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'purpose:credit_card',
'purpose:debt_consolidation',
'purpose:oth__med__vacation',
'purpose:major_purch__car__home_impr',
'initial_list_status:f',
'initial_list_status:w',
'term:36',
'term:60',
'emp_length:0',
'emp_length:1',
'emp_length:2-4',
'emp_length:5-6',
'emp_length:7-9',
'emp_length:10',
'mths_since_issue_d:<38',
'mths_since_issue_d:38-39',
'mths_since_issue_d:40-41',
'mths_since_issue_d:42-48',
'mths_since_issue_d:49-52',
'mths_since_issue_d:53-64',
'mths_since_issue_d:65-84',
'mths_since_issue_d:>84',
'int_rate:<9.548',
'int_rate:9.548-12.025',
'int_rate:12.025-15.74',
'int_rate:15.74-20.281',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'mths_since_earliest_cr_line:141-164',
'mths_since_earliest_cr_line:165-247',
'mths_since_earliest_cr_line:248-270',
'mths_since_earliest_cr_line:271-352',
'mths_since_earliest_cr_line:>352',
'delinq_2yrs:0',
'delinq_2yrs:1-3',
'delinq_2yrs:>=4',
'inq_last_6mths:0',
'inq_last_6mths:1-2',
'inq_last_6mths:3-6',
'inq_last_6mths:>6',
'open_acc:0',
'open_acc:1-3',
'open_acc:4-12',
'open_acc:13-17',
'open_acc:18-22',
'open_acc:23-25',
'open_acc:26-30',
'open_acc:>=31',
'pub_rec:0-2',
'pub_rec:3-4',
'pub_rec:>=5',
'total_acc:<=27',
'total_acc:28-51',
'total_acc:>=52',
'acc_now_delinq:0',
'acc_now_delinq:>=1',
'total_rev_hi_lim:<=5K',
'total_rev_hi_lim:5K-10K',
'total_rev_hi_lim:10K-20K',
'total_rev_hi_lim:20K-30K',
'total_rev_hi_lim:30K-40K',
'total_rev_hi_lim:40K-55K',
'total_rev_hi_lim:55K-95K',
'total_rev_hi_lim:>95K',
'annual_inc:<20K',
'annual_inc:20K-30K',
'annual_inc:30K-40K',
'annual_inc:40K-50K',
'annual_inc:50K-60K',
'annual_inc:60K-70K',
'annual_inc:70K-80K',
'annual_inc:80K-90K',
'annual_inc:90K-100K',
'annual_inc:100K-120K',
'annual_inc:120K-140K',
'annual_inc:>140K',
'dti:<=1.4',
'dti:1.4-3.5',
'dti:3.5-7.7',
'dti:7.7-10.5',
'dti:10.5-16.1',
'dti:16.1-20.3',
'dti:20.3-21.7',
'dti:21.7-22.4',
'dti:22.4-35',
'dti:>35',
'mths_since_last_delinq:Missing',
'mths_since_last_delinq:0-3',
'mths_since_last_delinq:4-30',
'mths_since_last_delinq:31-56',
'mths_since_last_delinq:>=57',
'mths_since_last_record:Missing',
'mths_since_last_record:0-2',
'mths_since_last_record:3-20',
'mths_since_last_record:21-31',
'mths_since_last_record:32-80',
'mths_since_last_record:81-86',
'mths_since_last_record:>=86',
]]

C:\Users\TAN\Anaconda3\lib\site-packages\pandas\core\indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [12]:
### Remove the dummy variable references to prevent from getting into dummy trap

In [13]:
# Here we store the names of the reference category dummy variables in a list.
ref_categories = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'delinq_2yrs:>=4',
'inq_last_6mths:>6',
'open_acc:0',
'pub_rec:0-2',
'total_acc:<=27',
'acc_now_delinq:0',
'total_rev_hi_lim:<=5K',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

In [14]:
inputs_train = inputs_train_with_ref_cat.drop(ref_categories,axis=1)

In [15]:
inputs_train.head(5)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86,mths_since_last_record:>=86
427211,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,NaN
206088,0,0,1,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,NaN
136020,1,0,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,NaN
412305,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,NaN
36159,0,0,1,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,NaN


In [26]:
inputs_train['mths_since_last_record:>=86'].head()

427211   NaN
206088   NaN
136020   NaN
412305   NaN
36159    NaN
Name: mths_since_last_record:>=86, dtype: float64

In [29]:
inputs_train = inputs_train.drop(['mths_since_last_record:>=86'],axis=1)

In [30]:
inputs_train.head(3)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NM_VA,addr_state:NY,...,dti:22.4-35,mths_since_last_delinq:Missing,mths_since_last_delinq:4-30,mths_since_last_delinq:31-56,mths_since_last_delinq:>=57,mths_since_last_record:Missing,mths_since_last_record:3-20,mths_since_last_record:21-31,mths_since_last_record:32-80,mths_since_last_record:81-86
427211,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0
206088,0,0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,0
136020,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [32]:
inputs_train.isna().sum()

grade:A                           0
grade:B                           0
grade:C                           0
grade:D                           0
grade:E                           0
                                 ..
mths_since_last_record:Missing    0
mths_since_last_record:3-20       0
mths_since_last_record:21-31      0
mths_since_last_record:32-80      0
mths_since_last_record:81-86      0
Length: 103, dtype: int64

In [33]:
loan_data_outputs_train.isna().sum()

1    0
dtype: int64

In [34]:
reg = LogisticRegression()

In [35]:
reg.fit(inputs_train,loan_data_outputs_train)

C:\Users\TAN\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\TAN\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [36]:
reg.coef_

array([[ 1.1673196 ,  0.91159641,  0.7092909 ,  0.51690723,  0.33212008,
         0.14216999,  0.08692764,  0.10177402,  0.01912996,  0.04666168,
         0.05001431,  0.05730008,  0.07417436,  0.12803031,  0.09083024,
         0.17305283,  0.23371337,  0.21053568,  0.2612607 ,  0.30947424,
         0.50037195,  0.09615706, -0.00771727,  0.30844668,  0.205042  ,
         0.21602708,  0.27087828,  0.04990461,  0.07436064,  0.10153669,
         0.12636127,  0.09714309,  0.06716362,  0.12806247,  1.03002309,
         0.82155544,  0.73747332,  0.5483558 ,  0.39875557,  0.16183319,
        -0.0629312 ,  0.84612352,  0.52712882,  0.28700105,  0.10478842,
         0.05628205,  0.04670583,  0.09264668,  0.12786517,  0.13370358,
         0.07927564,  0.05376715,  0.66605602,  0.52538898,  0.31573653,
         0.01597106, -0.09100767, -0.12170775, -0.13843962, -0.13700119,
        -0.13052753, -0.18881211,  0.12365678,  0.21630832, -0.01744106,
         0.03264954,  0.24842148,  0.04123182,  0.0

In [37]:
reg.intercept_

array([-1.23973398])